<a href="https://colab.research.google.com/github/krissme28/-studying_ML/blob/main/1_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Используем набор данных о ценах на жилье в Калифорнии https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html


In [8]:
#Загрузка набора данных California housing
data = fetch_california_housing()
X, y = data.data, data.target

In [14]:
df = pd.DataFrame(data.data, columns=data.feature_names)
print(df.head())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  


In [ ]:
#Нормализация данных
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
y = (y - y.min()) / (y.max() - y.min())

Создаем последовательную модель нейронной сети с двумя слоями Dense. Первый слой имеет 16 нейронов и функцию активации ReLU, а второй слой имеет 1 нейрон и функцию активации linear. Модель компилируется с оптимизатором 'adam' и функцией потерь 'mean_squared_error', затем обучается на данных.

In [ ]:
#Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Создание и обучение нейронной сети
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

1. `X_clean` и `y_clean`: Это обучающие данные без отравления (чистые данные). `X_clean` представляет собой матрицу признаков (входных данных), а `y_clean` - соответствующие метки классов.

2. `percent_poison`: Это процент отравления, который указывает, какую долю данных нужно "отравить" (изменить) для создания атаки.

3. `target_value`: Значение, которое устанавливаем для "отравленных" меток классов. (0.5)

4. `n_clean`: Количество чистых образцов в исходных данных.

5. `num_poison`: Количество образцов, которые нужно "отравить" в соответствии с заданным процентом отравления. Вычисляется как округленное значение отношения процента отравления к (1 - процент отравления).

6. `poison_indices`: Генерация случайных неповторяющихся индексов образцов для отравления.

7. Создание копий отравляющих образцов: Создание копии чистых данных с индексами из `poison_indices`.

8. Задание отравляющего значения: Метки классов в отравленных образцах (`y_poison`) устанавливаются в `target_value`.

9. Добавление отравленных образцов: Отравленные признаки и метки классов объединяются с чистыми данными для создания нового обучающего набора с "отравленными" образцами.

10. Возвращение отравленных данных: Функция возвращает новые "отравленные" образцы (`X_backdoor` и `y_backdoor`). Текст, выделенный полужирным шрифтом

In [ ]:
#Функция для проведения атаки отравления
def generate_backdoor(X_clean, y_clean, percent_poison, target_value):
    n_clean = X_clean.shape[0]
    num_poison = round((percent_poison * n_clean) / (1 - percent_poison))

    # Выбираем случайные индексы для отравления
    poison_indices = np.random.choice(n_clean, num_poison, replace=False)

    # Создаем копии отравляющих образцов
    X_poison = X_clean[poison_indices].copy()
    y_poison = y_clean[poison_indices].copy()

    # Задаем отравляющее значение
    y_poison[:] = target_value

    # Добавляем отравленные образцы к исходному обучающему набору
    X_backdoor = np.concatenate((X_clean, X_poison), axis=0)
    y_backdoor = np.concatenate((y_clean, y_poison), axis=0)

    return X_backdoor, y_backdoor

Epoch 1/50
465/465 [==============================] - 1s 2ms/step - loss: 0.0737 - val_loss: 0.0310
Epoch 2/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0255 - val_loss: 0.0259
Epoch 3/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0238 - val_loss: 0.0252
Epoch 4/50
465/465 [==============================] - 1s 2ms/step - loss: 0.0232 - val_loss: 0.0247
Epoch 5/50
465/465 [==============================] - 1s 2ms/step - loss: 0.0226 - val_loss: 0.0242
Epoch 6/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0221 - val_loss: 0.0238
Epoch 7/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0217 - val_loss: 0.0236
Epoch 8/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0214 - val_loss: 0.0230
Epoch 9/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0210 - val_loss: 0.0228
Epoch 10/50
465/465 [==============================] - 1s 1ms/step - loss: 0.0207 - val_loss: 0.0225

In [ ]:
# Преобразование меток в one-hot encoding
y_test_encoded = to_categorical(y_test, num_classes=2)

# Оцениваем производительность модели на тестовых данных до атаки
y_pred_before_attack = model.predict(X_test)
accuracy_before_attack = np.mean(np.argmax(y_pred_before_attack, axis=1) == np.argmax(y_test_encoded, axis=1))
print("Точность на тестовых данных до атаки: %.2f%%" % (accuracy_before_attack * 100))

129/129 [==============================] - 0s 895us/step
Точность на тестовых данных до атаки: 95.66%


In [ ]:
# Вызываем функцию атаки отравления с процентом отравления 10% и целевым значением 0.5
percent_poison = 0.1
target_value = 0.5
X_train_poisoned, y_train_poisoned = generate_backdoor(X_train, y_train, percent_poison, target_value)

# Обучаем модель на отравленных данных
model.fit(X_train_poisoned, y_train_poisoned, epochs=50, batch_size=32)

Epoch 1/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0209
Epoch 2/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0209
Epoch 3/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0208
Epoch 4/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0209
Epoch 5/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0209
Epoch 6/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0208
Epoch 7/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0207
Epoch 8/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0207
Epoch 9/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0206
Epoch 10/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0207
Epoch 11/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0207
Epoch 12/50
574/574 [==============================] - 1s 1ms/step - loss: 0.0206
Epoch 13/50
574/574 [====

In [ ]:
# Оцениваем производительность модели на тестовых данных с отравлением
mse_poisoned = model.evaluate(X_test, y_test, verbose=0)
print("MSE на тестовых данных с отравлением: %.4f" % mse_poisoned)

MSE на тестовых данных с отравлением: 0.0191


В данном примере, атака была проведена путем добавления "отравленных" образцов с целевым значением 0.5.